Global prediction of ABD using the trained model

In [3]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingRegressor
from pathlib import Path
import pickle
import re

Helper classes and functions.

In [2]:
class BGRBinaryEncoding(BaseEstimator, TransformerMixin):
    def __init__(self):
        self

    def fit(self, X , y=None):
        return self
    
    def transform(self, X):
        df = X.copy()

        df["bgr_tuple"] = [binary_encoding(t) for t in df.bgr]

        # df_bin = df['bgr_tuple'].apply(pd.Series, index=['bgr1','bgr2','bgr3'])

        df_bin=pd.DataFrame(df['bgr_tuple'].tolist())
        df_bin.columns=['bgr1', 'bgr2', 'bgr3']

        df = pd.concat([df,df_bin],axis='columns')

        df = df.dropna()

        df = df.drop(["bgr_tuple","bgr"],axis="columns")

        return df
    
# custom transformer for sklearn pipeline
class ColumnExtractor(TransformerMixin, BaseEstimator):
    def __init__(self, cols):
        self.cols = cols

    def transform(self, X):
        return X[self.cols]

    def fit(self, X, y=None):
        return self    
    
class LogarithmizeWaterObservables(BaseEstimator, TransformerMixin):
    def __init__(self, cols):
        self.cols = cols
    def fit(self, X , y=None):
        return self
    def transform(self, X):
        df = X.copy()
        log_preds = log_predictors(self.cols)
        df[log_preds] = np.log(df[log_preds]+1)     
        return df    

def keep_predictor(plist,pname,pval):
    if pval>0.5:
        plist.append(pname)
    return plist


def build_predictor_list(predictor_dict):
    plist = []
    for predictor in predictor_dict:
        plist = keep_predictor(plist,predictor,predictor_dict[predictor])
    return plist    

def func(x):
    return np.log(x+1)

def inverse_func(x):
    return np.exp(x)-1

def binary_encoding(bgr):
    match bgr:
        case "Palearctic":
            return (0,0,0)
        case "Indomalayan":
            return (0,1,0)
        case "Australasia":
            return (0,0,1)
        case "Nearctic":
            return (0,1,1)
        case "Afrotropic":
            return (1,0,0)
        case "Neotropic":
            return (1,1,0)                    

log_transformer = FunctionTransformer(func=func,inverse_func=inverse_func)

def log_predictors(predictor_list):
    all = ["yp","pwm","pet","ps","pcq","pdq","pwaq","pweq"]
    return [f for f in all if f in predictor_list]


def predictor_list(predictor_string):
    pred = predictor_string.split("-")
    return pred 

NameError: name 'BaseEstimator' is not defined

Load model.

In [5]:
pickled_model = "/home/dibepa/git/global.agb.ml/data/training/detailed.allometries.model/abd_model_bioclim_notrees_noweights.pkl"

with open(pickled_model, 'rb') as f:
    model = pickle.load(f)

Iterate over predictor windows  and make ABD estimation.

In [6]:
datadir = "/home/dibepa/git/global.agb.ml/data/predict/predictor.data.onlybioclim/"

for i,file in enumerate(Path(datadir).glob('*')):

    window = re.findall("predictors_global_data(.*)",file.name)[0]
    print(window)

    filename = "/home/dibepa/git/global.agb.ml/data/predict/detailed.allometries.model/window.prediction/ABD{}".format(window)

    if not Path(filename).is_file(): 

        prediction_data = pd.read_csv(file)

        # prediction_data = prediction_data.drop(prediction_data[(prediction_data.bgr == "Oceania")|(prediction_data.bgr == "Antarctica")].index)

        if len(prediction_data.index)>0:
    
            coords = prediction_data[["x","y"]]
            X = prediction_data.drop(["x","y"],axis="columns")

            X.replace([np.inf, -np.inf], np.nan, inplace=True)
            X=X.reset_index(drop=True)

            try:

                abd= model.predict(X)

                abd_df = pd.DataFrame({"abd":abd})
                
                predicted = pd.concat([abd_df,coords], axis = "columns")

                predicted.to_csv(filename,index=False)

            except:
                print("Could not make prediction.")
                continue

_lon_18.0_lat_9.0.csv


/home/dibepa/miniconda3/envs/tree.cluster/lib/python3.12/site-packages/xgboost/data.py:312: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/dibepa/miniconda3/envs/tree.cluster/lib/python3.12/site-packages/xgboost/data.py:314: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  elif is_categorical_dtype(dtype) and enable_categorical:
/home/dibepa/miniconda3/envs/tree.cluster/lib/python3.12/site-packages/xgboost/data.py:345: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype)
/home/dibepa/miniconda3/envs/tree.cluster/lib/python3.12/site-packages/xgboost/data.py:336: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dt

Could not make prediction.
_lon_36.0_lat_27.0.csv
Could not make prediction.
_lon_-162.0_lat_63.0.csv
